# Automate Grading AMATH 301
The purpose of this notebook is to combine grades from Matlab Grader and Gradescope. It will then add the scores to the canvas gradebook CSV to import. Certain lines will need to be edited each time it is used. 

## Things to edit: 
There are comments next to the things that need to be changed for each grading process. 
1. Change the path to the csv files to match where they are saved on your computer. 
2. Depending on the number of matlab grader problems, you will need to add additional columns. Also you can adjust the weight of the matlab grader tests. (Usually 5)
3. Add the number of Matlab grader questions. This will be used to calculate the column of the total score. 
4. Add additional merge statements for the number of questions and add them to the total score column.
5. Look at the exported canvas CSV and determine the index of the column that needs edited (recalling that python indexes at zero) 
6. *** Make sure that you change the approprate column headers to (READ ONLY) before importing to canvas ***
7. Save the CSV in the approprate place on your computer. 

### Debugging Notes: 
1. Make sure in references that there are no spaces in front of the ID numbers.
2. You can display the references data frame so you can see what is needed for it.

In [1]:
import pandas as pd;
#Change the path to the csv files to match where they are saved on your computer. 
mat_grader = pd.read_csv('/Users/shannondow/Desktop/301Automate/MatlabGrader_hw1.csv')
gradescope = pd.read_csv('/Users/shannondow/Desktop/301Automate/Gradescope_hw1.csv')
canvas = pd.read_csv('/Users/shannondow/Desktop/301Automate/Canvas_hw1.csv')
reference = pd.read_csv('/Users/shannondow/Desktop/301Automate/References.csv')

In [2]:
mat_grader = mat_grader.sort_values(by = ['Student Email'])
mat_grader = mat_grader[[ 'Student Email', 'Tests Passed','Total Tests','Problem Title']]
gradescope = gradescope[['SID','Total Score']]
gradescope = gradescope.rename(columns = {'Total Score': 'GS Tot'})
reference = reference.rename(columns = {'StudentNo': 'SID'})
reference = reference.rename(columns = {'Email':'Student Email'})

In [3]:
#Depending on the number of gradescope problems, repeat these lines: 
#Change the name of HW1 to the number HW it is
mg1 = mat_grader[mat_grader['Problem Title'] == 'HW1: Problem 1']
mg1 = mg1.rename(columns = {'Tests Passed':'mg_q1'})
mg1 = mg1[[ 'Student Email', 'mg_q1']]
mg2 = mat_grader[mat_grader['Problem Title'] == 'HW1: Problem 2'] 
mg2 = mg2.rename(columns = {'Tests Passed':'mg_q2'})
mg2 = mg2[[ 'Student Email', 'mg_q2']]
#Define the weight of each gradescope problem and the number of questions:
num_mg_questions = 2
mgw = 5

In [4]:
#Add an additional merge statement for each matlab grader question. 
reference = reference.merge(mg1, on = 'Student Email')
reference = reference.merge(mg2, on = 'Student Email')
# Add another column(s) if there are there
reference['mg_tot'] = (reference['mg_q1']+ reference['mg_q2'])*mgw

In [5]:
gradescope['SID'] = gradescope['SID'].astype(str)
reference['SID'] = reference['SID'].astype(str)
reference = reference.merge(gradescope, on = 'SID')
reference['TOTAL'] = (reference['mg_tot']+ reference['GS Tot'])

In [6]:
# This is the column of the homework that needs edited (Look at the CSV and recall that 
# python begins its indexing at zero
col_edit = 6;
i = 2;
while i < canvas.shape[0]:
    canvas.iloc[i,col_edit] = 0
    i = i+1
i = 2
while i < canvas.shape[0]:
    for j in range(0,reference.shape[0]):
        if canvas.iloc[i,3] == reference.iloc[j,1]:
            canvas.iloc[i,col_edit] = reference.iloc[j,(7+num_mg_questions)]
    i = i+1


In [7]:
canvas.head()

,Student,ID,SIS User ID,SIS Login ID,Section,Homework 0 (5327434),Homework 1 (5327437),Homework 2 (5431255),Activity 1 (5327424),Beginning of Quarter Survey (5327426),...,Final Exam Final Score,Final Exam Unposted Final Score,Imported Assignments Current Score,Imported Assignments Unposted Current Score,Imported Assignments Final Score,Imported Assignments Unposted Final Score,Current Score,Unposted Current Score,Final Score,Unposted Final Score
0,NaN,NaN,NaN,NaN,NaN,Manual Posting,Manual Posting,Manual Posting,Manual Posting,Manual Posting,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Points Possible,NaN,NaN,NaN,NaN,0,100,100,10,3,...,(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only)
2,"Felten, Anna",1234563.0,796C2D5CA1DB557115E7629EBD314831,feltan,AMATH 301 B,NaN,93,NaN,10,3,...,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20
3,"Ellis, Andrew",9922782.0,796C2D5CA1DB557115E7629EBD314831,drewb,AMATH 301 B,NaN,76,NaN,10,3,...,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20
4,"Tubert, Nik",1836593.0,796C2D5CA1DB557115E7629EBD314831,skimnik,AMATH 301 B,NaN,95,NaN,10,3,...,NaN,NaN,NaN,NaN,NaN,NaN,100,100,13.94,20


In [8]:
#Save the updated CSV to a specified location with a desired name: 
canvas.to_csv('/Users/shannondow/Desktop/301Automate/HW1_update.csv')